In [ ]:
import json

from prettytable import PrettyTable
from stringMatcher import levenshtein_ratio_and_distance
from time import time

table = PrettyTable(field_names=["UserID", "Rating", "Tags", "Timestamp"])

## Load The database into the memory...

In [ ]:
data = json.load(open("dataStore/dataFinal.json", "rb"))

## Load the Global Secondary index in the memory...

In [ ]:
GIS = json.load(open("dataStore/dataFinal_GIS.json", "rb"))

## Write a function to search the movies by ```Title```

In [ ]:
def getClosestMatch(queryString):
    # Initialize the time counter...
    tic                  = time()
    
    # First Error Handling
    # Manipulating query string
    def properCase(queryString):
        words=queryString.split()
        temp=''
        for i in words:
            temp=temp+" "+(i.capitalize())
        return temp

    # Find closest match of the queryString from the Global Secondary Index...
    matchRatios          = [levenshtein_ratio_and_distance(properCase(queryString), KEY) for KEY, _ in GIS.items()]
    bestMatchRatio       = max(matchRatios)

    # If the best match ratio is less than 50% then we will assume that the records doesn't exist...
    if (bestMatchRatio < 0.5):
        return {
            "response code" : 404,
            "search time"   : f"{time()-tic} seconds",
            "message"       : "error",
            "response body" : "no match found"
        }
    
    # Otherwise...
    bestMatchRatio_index = matchRatios.index(bestMatchRatio)

    # Find the movieID of the movie which is the best match to the queryString...
    bestMatch_MovieTitle = list(GIS)[bestMatchRatio_index]
    bestMatch_MovieID    = GIS[bestMatch_MovieTitle]

    # Now we have movieID which is the primary key in our main data-store...
    # Retrieve the details of the movie from the MovieID
    movieInfo            = data[str(bestMatch_MovieID)]
    
    # Mark the search completion time...
    toc                  = time()

    # Add Movie Title to the response...
    movieInfo["title"] = bestMatch_MovieTitle
    # Construct the response Model...
    responseModel = {
        "response code" : 200,
        "search time"   : f"{toc-tic} seconds",
        "message"       : "success",
        "response body" : movieInfo
    }
    
    return responseModel

## Write a function to make a request and visualize the data returned by the search function...

In [ ]:
def request(movieToSearch=""):
    # Movie name should always be in a string...
    movieToSearch = str(movieToSearch)
    # Check if the movie name is valid...
    if(movieToSearch.replace(" ", "").strip() == ""):
        return f"Invalid Movie Name"
    
    # Make a request to fetch the movie info...
    response = getClosestMatch(queryString = movieToSearch)

    # Check the response code...
    if response["response code"] == 404:
        return f"ERROR: No records found"
    
    # otherwise...
    # Construct user ratings table...
    ratings = response["response body"]["user_rating"]
    tags    = response["response body"]["tags"]

    for rating in ratings:
        try    : tags_by_a_single_user_to_a_single_movie = ", ".join([tag["tag"] for tag in tags if rating["userId"] == tag["userId"]]) # Please don't mind the long veriable name :-)
        except : tags_by_a_single_user_to_a_single_movie = ""
        break
    table.add_row([rating["userId"], rating["rating"], tags_by_a_single_user_to_a_single_movie, rating["time_stamp"]])

    #Second Error
    #To calculate average movie rating
    sum_of_ratings=0.0
    no_of_ratings=0
    for rating in ratings:
        sum_of_ratings=sum_of_ratings+rating["rating"]
        no_of_ratings+=1
    avg_rating=round(sum_of_ratings/no_of_ratings,1)

    # Now we need to check if a user has given 
    data_to_print = f"\
    Title          : {response['response body']['title']}\n\
    Genre          : {response['response body']['genre']}\n\
    User Ratings   : {avg_rating}/5.0\n\
    "
    print(data_to_print)
    print(table)
    table.clear_rows()

## Test Cases

In [ ]:
request(movieToSearch="Jumanji")

In [ ]:
request(movieToSearch="jUMANJI")

In [ ]:
request(movieToSearch="HITMAN")

In [ ]:
request(movieToSearch="GOLDEN COMPASS")

In [ ]:
request(movieToSearch="gOLDEN cOMPASS")

In [ ]:
request(movieToSearch="goldencompass")

In [ ]:
request(movieToSearch="gONE bABY gONE")

In [ ]:
request(movieToSearch="GONEBABYGONE")

In [ ]:
request(movieToSearch="XYZ")

In [ ]:
request(movieToSearch="daninreallife")

In [ ]:
request(movieToSearch="DAN IN REAL LIFE")

In [ ]:
request(movieToSearch="other diseases")

In [ ]:
request(movieToSearch="love and")

In [ ]:
request(movieToSearch="tAXI 4")

In [ ]:
request(movieToSearch="VALET, The (La doublure)")

In [ ]:
request(movieToSearch="valet, The (la DOUbLure)")

In [ ]:
request(movieToSearch="Jonestown: THE LIFE AND DEATH of PEOPLES TEMPLE")

In [ ]:
request(movieToSearch="JonestowN")

In [ ]:
request(movieToSearch="It's a VERY MERRY MUPET ChristMAS Movie")

In [ ]:
request(movieToSearch="Muppet Christmas Movie")